In [4]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
TRAIN_PATH = 'G:/Kaggle_data/Kaggle_bowl/stage1_train/'
TEST_PATH = 'G:/Kaggle_data/Kaggle_bowl/stage1_test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

smooth = 1

# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask
    
#def scale_img_canals(an_img):
for i in range(IMG_CHANNELS):
    canal = img[:,:,i]
    canal = canal - canal.min()
    canalmax = canal.max()
    if canalmax > 0:
        factor = 255/canalmax
        canal = (canal * factor).astype(int)
    img[:,:,i] = canal

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img
    
for i in range(IMG_CHANNELS):
    canal = img[:,:,i]
    canal = canal - canal.min()
    canalmax = canal.max()
    if canalmax > 0:
        factor = 255/canalmax
        canal = (canal * factor).astype(int)
    img[:,:,i] = canal

print('Done!')

# Check if training data looks all right
ix = random.randint(0, len(train_ids))
imshow(X_train[ix])
plt.show()
imshow(np.squeeze(Y_train[ix]))
plt.show()

# Define IoU metric
"""def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)"""

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
model.summary()

# Fit model
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=50, 
                    callbacks=[earlystopper, checkpointer])

# Predict on train, val and test
model = load_model('model-dsbowl2018-1.h5', custom_objects={'dice_coef': dice_coef})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))
    
# Perform a sanity check on some random training samples
# ix = random.randint(0, len(preds_train_t))
# imshow(X_train[ix])
# plt.show()
# imshow(np.squeeze(Y_train[ix]))
# plt.show()
# imshow(np.squeeze(preds_train_t[ix]))
# plt.show()

# Perform a sanity check on some random validation samples
# ix = random.randint(0, len(preds_val_t))
# imshow(X_train[int(X_train.shape[0]*0.9):][ix])
# plt.show()
# imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))
# plt.show()
# imshow(np.squeeze(preds_val_t[ix]))
# plt.show()

# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)
        
new_test_ids = []
rles = []
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds_test_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))
    
# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('sub-dsbowl2018-1_17.03.2018_dice_256_scale.csv', index=False)



D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Getting and resizing train images and masks ... 


100%|██████████| 670/670 [02:37<00:00,  4.26it/s]


Getting and resizing test images ... 


100%|██████████| 65/65 [00:01<00:00, 54.90it/s]


...

603/603 [==============================] - ETA: 5:02 - loss: 0.0752 - dice_coef: 0.816 - ETA: 4:50 - loss: 0.0664 - dice_coef: 0.854 - ETA: 4:43 - loss: 0.0709 - dice_coef: 0.857 - ETA: 4:34 - loss: 0.0697 - dice_coef: 0.854 - ETA: 4:26 - loss: 0.0674 - dice_coef: 0.859 - ETA: 4:17 - loss: 0.0695 - dice_coef: 0.857 - ETA: 4:09 - loss: 0.0671 - dice_coef: 0.863 - ETA: 4:01 - loss: 0.0675 - dice_coef: 0.865 - ETA: 3:53 - loss: 0.0680 - dice_coef: 0.865 - ETA: 3:45 - loss: 0.0678 - dice_coef: 0.864 - ETA: 3:37 - loss: 0.0716 - dice_coef: 0.863 - ETA: 3:28 - loss: 0.0701 - dice_coef: 0.864 - ETA: 3:20 - loss: 0.0727 - dice_coef: 0.862 - ETA: 3:13 - loss: 0.0734 - dice_coef: 0.862 - ETA: 3:06 - loss: 0.0738 - dice_coef: 0.860 - ETA: 2:57 - loss: 0.0760 - dice_coef: 0.856 - ETA: 2:49 - loss: 0.0782 - dice_coef: 0.857 - ETA: 2:41 - loss: 0.0764 - dice_coef: 0.859 - ETA: 2:33 - loss: 0.0780 - dice_coef: 0.859 - ETA: 2:25 - loss: 0.0776 - dice_coef: 0.861 - ETA: 2:17 - loss: 0.0770 - dice_coef:

603/603 [==============================] - ETA: 5:30 - loss: 0.0821 - dice_coef: 0.811 - ETA: 5:16 - loss: 0.0780 - dice_coef: 0.812 - ETA: 5:02 - loss: 0.0736 - dice_coef: 0.840 - ETA: 4:52 - loss: 0.0683 - dice_coef: 0.850 - ETA: 4:41 - loss: 0.0775 - dice_coef: 0.843 - ETA: 4:32 - loss: 0.0756 - dice_coef: 0.843 - ETA: 4:22 - loss: 0.0729 - dice_coef: 0.851 - ETA: 4:13 - loss: 0.0730 - dice_coef: 0.853 - ETA: 4:05 - loss: 0.0717 - dice_coef: 0.854 - ETA: 3:56 - loss: 0.0744 - dice_coef: 0.857 - ETA: 3:47 - loss: 0.0731 - dice_coef: 0.859 - ETA: 3:39 - loss: 0.0731 - dice_coef: 0.858 - ETA: 3:30 - loss: 0.0751 - dice_coef: 0.859 - ETA: 3:22 - loss: 0.0763 - dice_coef: 0.855 - ETA: 3:13 - loss: 0.0741 - dice_coef: 0.858 - ETA: 3:05 - loss: 0.0744 - dice_coef: 0.858 - ETA: 2:56 - loss: 0.0749 - dice_coef: 0.859 - ETA: 2:48 - loss: 0.0761 - dice_coef: 0.858 - ETA: 2:39 - loss: 0.0758 - dice_coef: 0.858 - ETA: 2:30 - loss: 0.0754 - dice_coef: 0.858 - ETA: 2:22 - loss: 0.0774 - dice_coef:

603/603 [==============================] - ETA: 5:11 - loss: 0.0670 - dice_coef: 0.852 - ETA: 5:05 - loss: 0.0645 - dice_coef: 0.861 - ETA: 4:56 - loss: 0.0627 - dice_coef: 0.864 - ETA: 4:47 - loss: 0.0711 - dice_coef: 0.852 - ETA: 4:37 - loss: 0.0738 - dice_coef: 0.852 - ETA: 4:29 - loss: 0.0687 - dice_coef: 0.857 - ETA: 4:20 - loss: 0.0671 - dice_coef: 0.858 - ETA: 4:12 - loss: 0.0664 - dice_coef: 0.859 - ETA: 4:03 - loss: 0.0710 - dice_coef: 0.859 - ETA: 3:55 - loss: 0.0699 - dice_coef: 0.856 - ETA: 3:46 - loss: 0.0691 - dice_coef: 0.861 - ETA: 3:38 - loss: 0.0675 - dice_coef: 0.863 - ETA: 3:29 - loss: 0.0686 - dice_coef: 0.860 - ETA: 3:20 - loss: 0.0705 - dice_coef: 0.858 - ETA: 3:12 - loss: 0.0704 - dice_coef: 0.858 - ETA: 3:03 - loss: 0.0714 - dice_coef: 0.857 - ETA: 2:55 - loss: 0.0699 - dice_coef: 0.857 - ETA: 2:46 - loss: 0.0703 - dice_coef: 0.857 - ETA: 2:38 - loss: 0.0706 - dice_coef: 0.859 - ETA: 2:29 - loss: 0.0695 - dice_coef: 0.861 - ETA: 2:21 - loss: 0.0697 - dice_coef:

603/603 [==============================] - ETA: 5:07 - loss: 0.0750 - dice_coef: 0.844 - ETA: 5:02 - loss: 0.0682 - dice_coef: 0.860 - ETA: 4:53 - loss: 0.0725 - dice_coef: 0.858 - ETA: 4:45 - loss: 0.0709 - dice_coef: 0.861 - ETA: 4:36 - loss: 0.0708 - dice_coef: 0.863 - ETA: 4:32 - loss: 0.0670 - dice_coef: 0.867 - ETA: 4:24 - loss: 0.0678 - dice_coef: 0.869 - ETA: 4:15 - loss: 0.0664 - dice_coef: 0.875 - ETA: 4:06 - loss: 0.0659 - dice_coef: 0.873 - ETA: 3:57 - loss: 0.0660 - dice_coef: 0.873 - ETA: 3:48 - loss: 0.0695 - dice_coef: 0.871 - ETA: 3:39 - loss: 0.0676 - dice_coef: 0.872 - ETA: 3:31 - loss: 0.0674 - dice_coef: 0.873 - ETA: 3:22 - loss: 0.0679 - dice_coef: 0.871 - ETA: 3:13 - loss: 0.0662 - dice_coef: 0.871 - ETA: 3:05 - loss: 0.0657 - dice_coef: 0.872 - ETA: 2:56 - loss: 0.0647 - dice_coef: 0.872 - ETA: 2:48 - loss: 0.0681 - dice_coef: 0.871 - ETA: 2:39 - loss: 0.0677 - dice_coef: 0.871 - ETA: 2:30 - loss: 0.0681 - dice_coef: 0.871 - ETA: 2:21 - loss: 0.0696 - dice_coef:

TypeError: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.

In [6]:
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)
        
new_test_ids = []
rles = []
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds_test_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))
    
# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('sub-dsbowl2018-1_17.03.2018_dice_256_scale.csv', index=False)